In [ ]:
try:    importlib.reload(Jupytils)
except: import Jupytils
showTopbar("Crystal Ball Examples", menu=[("Help", "#"), ("Home", "#")])

import pymc3 as pymc3
from pymc3 import *
from numpy import median
from scipy import optimize
import openpyxl

In [ ]:
#Excel functions

def SUM(ran):
    a = []
    for c in ws[ran]:
        for d in c:
            a.append(d.value)
    return np.array(a).sum()  

def IFP(fun):
    f1 = fun.split(',')
    lm ='''({}) if ({}) else ({})'''.format(f1[1], f1[0], f1[2]);
    return (lm)

def IF(fun):
    lm = IFP(fun)
    return eval(lm)

funcs = ['SUM', 'IF']
def LAMBDA(ss):
    s = ss.strip();
    s = s[1:]
    fw = re.match(r'(\w+)', s).group()
    if (fw in funcs):
        ret = fw + "('" + s[len(fw)+1:-1] + "')" 
    else:
        ret =  ss[1:]
        
    return ret
    
#s ='=SUM(B2:B4)'
#LAMBDA(s)

def PYMC3(cell, func):
    f = [f.strip() for f in func.split(',')]
    ret= ("{:4s} = {} ".format(cell , func) )
    
    return ret;

In [ ]:
import openpyxl
import numbers

# Helper function to define excel spreadsheet variables and lambda functions

file="cb2.xlsx"
wb = openpyxl.load_workbook(file)
ws =wb.get_active_sheet()
rows = len(ws.row_dimensions)
cols = len(ws.column_dimensions)
#We have everything defined in Column B, therefore create those variables

g = globals()

model = "";
assus = "";
vaars = "";

for r in range(1,rows):
    for c in range(1,cols):
        cell= ws.cell(row=r, column=c);
        v = cell.value
        if ( type(v) == str and v.startswith("=")):
            nv =  v.strip()
            pv = LAMBDA(nv)
            model += ("{:4s}= lambda : {}\n".format(cell.coordinate, pv) )
            continue;
            
        if ( type(v) == str and v.strip().lower().find("pymc3") >= 0 ):
            nv =  v.strip()
            pv = PYMC3( cell.coordinate, nv)
            assus += ("{} {}\n".format('', pv) )
            continue;
            
        if ( v is None or not isinstance(v, numbers.Number)):
            continue
        g[cell.coordinate] = v
        vaars += ( "{:4s}= {:6}    #defined value {}\n".format(cell.coordinate, v, type(v)))

print (assus, "\n\n", model, "\n\n", vaars, sep='')

In [ ]:
IF('B9,B19()-B5(),-B2-B3')

In [ ]:
determs = ["Gross Profit", "Net Profit"];
with pymc3.Model() as mcmc:
    B5  = lambda : SUM('B2:B4')
    B9  = lambda : B8>=20
    B14 = lambda : B12*(1+B13)
    B19 = lambda : B14()*B17*B18
    B21 = lambda : IF('B9,B19()-B5(),-B2-B3')
    
    B3   = pymc3.Uniform('Testing Cost', 3,5)
    B4   = Triangular("Marketing Cost", 12,18,16) #Pymc3 
    B8   = pymc3.Binomial("Patients Cured", 100, .25) 
    #C13  = Custom("Growth Rate", 0,5, 0.75, -15, 5, 0.25) #Pymc3 
    B17  = pymc3.Normal("Market Penetration",.08, .02) 
    
    C19  = pymc3.Deterministic("Gross Profit",B19() ) 
    C21  = pymc3.Deterministic("Net Profit", B21()) 

    #m1 = tt.printing.Print("Gross Profit",)(B3)
    #m2 = tt.printing.Print("Net Profit")(B4)
    
    step = pymc3.Metropolis()
    traced = sample(30000, step)



In [ ]:
nn = len(traced.varnames);
plt.figure(figsize=(20,6))

cc = 'ymbgrcmykw'*5;
for i, j in enumerate(traced.varnames):
    mn = np.mean(traced[j])
    nn = "{}{}{}".format(1+int(len(traced.varnames)/4),4, i+1) ;
    print (i, j, nn)
    
    if (j in determs):
        ablines=['mean', 'std']; legend=True; low=4; high=np.math.inf
    else:
        ablines=[]; legend=False; low=None; high=None
        
    plthist(traced[j], title=j+' $\mu$='+ str(mn), ylabel='Frequency', xlabel=j+' $', facecolor=cc[i], subplot=nn, ablines=ablines, legend=legend, low=low, high=high);

plt.subplots_adjust(wspace=None, hspace=.3)



In [ ]:
from pymc3 import distributions
from pymc3.distributions.dist_math import *
import theano.tensor as tt

class Triangular(Continuous):
    """
    Continuous Triangular log-likelihood
    Implemented by J. A. Fonseca 22/12/16
    Parameters
    ----------
    lower : float
        Lower limit.
    c: float
        mode
    upper : float
        Upper limit.
    """

    def __init__(self, lower=0, upper=1, c=0.5,
                 *args, **kwargs):
        super(Triangular, self).__init__(*args, **kwargs)

        self.c = c
        self.lower = lower
        self.upper = upper
        self.mean = c
        self.median = self.mean

    def random(self, point=None, size=None):
        c, lower, upper = draw_values([self.c, self.lower, self.upper],
                                      point=point)
        return generate_samples(stats.triang.rvs, c=c-lower, loc=lower, scale=upper-lower,
                                size=size, dist_shape=self.shape, random_state=None)

    def logp(self, value):
        c = self.c
        lower = self.lower
        upper = self.upper
        return tt.switch(alltrue_elemwise([lower <= value, value < c]),
                         tt.log(2 * (value - lower) / ((upper - lower) * (c - lower))),
                         tt.switch(tt.eq(value, c), tt.log(2 / (upper - lower)),
                         tt.switch(alltrue_elemwise([c < value, value <= upper]),
                         tt.log(2 * (upper - value) / ((upper - lower) * (upper - c))),np.inf)))

In [ ]:
def _isin(l=0,u=4.5, n=4.6):
    if (n >=l and n <=u):
        return True;
    else:
        False
        


In [ ]:
ranges=[(1,2), (3,4)]
probs=[.25,.75]

#ASSERT Validaity
if ( 1 != sum(probs) ):
    print ("Probs not equal 1")

probs.sort()
rrs = []
l=0
for j in probs:
    rrs.append((l, l+j))
    l = j
print(rrs, ranges)
#--------------------------------------------    
def isIn(r, ranges):
    p=np.random.rand()
    #print(r, p, ranges)
    for i,j in enumerate(r):
        l=j[0]
        u=j[1]
        if ( p >= l and p < u):
            return i,ranges[i]
    return None, None

vals=[];
for i in range(10):
    ii,ri = isIn(rrs, ranges)
    p= np.random.rand()
    
    l = ri[0] * (1+p);
    u = ri[1] * (1+p);

    print("Now: ",ri, l,u, p)

    pp= np.random.uniform(low=ri[0], high=ri[1])

    
    if (l >= ri[0] and u <= ri[1]):
        vals.append(ri)
    else:
        vals.append(0)

print(vals)


In [ ]:
from pymc3 import distributions
from pymc3.distributions.dist_math import *
import theano.tensor as tt

class Triangular1(Continuous):
    def __init__(self, lower=0, upper=1, c=0.5,
                 *args, **kwargs):
        super(Triangular1, self).__init__(*args, **kwargs)

        self.c = c
        self.lower = lower
        self.upper = upper
        self.mean = c
        self.median = self.mean
        print("__init__ Called with: ", lower, upper, c, *args, **kwargs)

    def random(self, point=None, size=None):
        c, lower, upper = draw_values([self.c, self.lower, self.upper],
                                      point=point)
        ret = generate_samples(stats.triang.rvs, c=c-lower, loc=lower, scale=upper-lower,
                                size=size, dist_shape=self.shape, random_state=None)
        
        print("random(): Called with: ", point, size, c, lower, upper, ret)
        return ret
    
    def logp(self, value):
        c = self.c
        lower = self.lower
        upper = self.upper
        ret = tt.switch(alltrue_elemwise([lower <= value, value < c]),
                         tt.log(2 * (value - lower) / ((upper - lower) * (c - lower))),
                         tt.switch(tt.eq(value, c), tt.log(2 / (upper - lower)),
                         tt.switch(alltrue_elemwise([c < value, value <= upper]),
                         tt.log(2 * (upper - value) / ((upper - lower) * (upper - c))),np.inf)))
    
        print("logp(): Called with: ", value, ret)
        return ret;


In [ ]:
def D1(v):
    ret = v+1;
    print ("I am called with: ", v, ret)
    return ret

with pymc3.Model() as mcmc1:
    TT   = Triangular1("Marketing Cost", 12,18,16) #Pymc3     
    C21  = pymc3.Deterministic("Net Profit", D1(TT) )

    
    step = pymc3.Metropolis()
    traced = sample(1000, step)

nn = len(traced.varnames);
plt.figure(figsize=(20,6))

cc = 'ymbgrcmykw'*5;
for i, j in enumerate(traced.varnames):
    mn = np.mean(traced[j])
    nn = "{}{}{}".format(1+int(len(traced.varnames)/4),4, i+1) ;
    print (i, j, nn)
    
    if (j in determs):
        ablines=['mean', 'std']; legend=True; low=4; high=np.math.inf
    else:
        ablines=[]; legend=False; low=None; high=None
        
    plthist(traced[j], title=j+' $\mu$='+ str(mn), ylabel='Frequency', xlabel=j+' $', facecolor=cc[i], subplot=nn, ablines=ablines, legend=legend, low=low, high=high);

plt.subplots_adjust(wspace=None, hspace=.3)

In [ ]:
#TT   = Triangular1("Marketing Cost", 12,18,16) #Pymc3     
